In [2]:
import nltk
import io
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from pymystem3 import Mystem
from string import punctuation
import pymorphy2
from nltk.stem.snowball import SnowballStemmer 
import math

In [3]:
def GetWordsFromFile(fileName):
    
    with open(fileName, "r", errors='ignore', encoding='utf-8') as file:
        documents = file.read()
    words = nltk.word_tokenize(documents)
    
    return words

In [4]:
def GetSentencesFromFile(fileName):
    
    with open(fileName, "r", errors='ignore', encoding='utf-8') as file:
        documents = file.read()
    sentences = nltk.sent_tokenize(documents)
    
    return sentences

In [5]:
def CreateStemmingFrequencyDict(fileInputName, fileOutputName):
    
    words = GetWordsFromFile(fileInputName)
    stopWordsArray = stopwords.words("russian")
    stemmingFrequencyDict = {}
    stemmer = SnowballStemmer("russian")
    punctuationList = punctuation+'«»—„“–'
    
    for i in range(len(words)):
        oldWord = words[i].lower()
        if not (oldWord == " " or oldWord == "..." or oldWord.strip() in punctuationList):
            newWord = stemmer.stem(words[i].lower())
            if not (newWord in stopWordsArray):
                if (newWord in stemmingFrequencyDict):
                    stemmingFrequencyDict[newWord]+=1
                else:
                    stemmingFrequencyDict[newWord]=1
                    
    with open(fileOutputName, "w", errors='ignore', encoding='utf-8') as file:
        for k in sorted(stemmingFrequencyDict, key=stemmingFrequencyDict.get, reverse=True):
            file.write(k + " " + str(stemmingFrequencyDict[k])+"\n")
            
    return stemmingFrequencyDict

In [6]:
def CreateLemmatisationFrequencyDict(fileInputName, fileOutputName):
    
    words = GetWordsFromFile(fileInputName)
    stopWordsArray = stopwords.words("russian")
    frequencyDict = {}
    morph = pymorphy2.MorphAnalyzer()
    punctuationList = punctuation+'«»—„“–'
    
    for i in range(len(words)):
        oldWord = words[i].lower()
        if not (oldWord == " " or oldWord == "..." or oldWord.strip() in punctuationList):
            newWord = morph.parse(oldWord)[0].normal_form
            if not (newWord in stopWordsArray):
                if (newWord in frequencyDict):
                    frequencyDict[newWord]+=1
                else:
                    frequencyDict[newWord]=1
    
    with open(fileOutputName, "w", errors='ignore', encoding='utf-8') as file:
        for k in sorted(frequencyDict, key=frequencyDict.get, reverse=True):
            file.write(k + " " + str(frequencyDict[k])+"\n")
            
    return frequencyDict

In [7]:
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('punkt')

In [8]:
sourceFileNames = {
    "Слово о полку.txt":0,
    "Поучение Мономаха.txt":1,
    "Плач Иеремии.txt":2,
    "Исповедь Круга.txt":3
}

## Читаем Слово о полку Игореве

In [9]:
dictStemming1=CreateStemmingFrequencyDict("Слово о полку.txt", "S1.txt")

In [10]:
dictLemmatisation1=CreateLemmatisationFrequencyDict("Слово о полку.txt","L1.txt")

## Читаем поучение Мономаха

In [11]:
dictStemming2=CreateStemmingFrequencyDict("Поучение Мономаха.txt", "S2.txt")

In [12]:
dictLemmatisation2=CreateLemmatisationFrequencyDict("Поучение Мономаха.txt","L2.txt")

## Читаем Плач Иеремии

In [13]:
dictStemming3=CreateStemmingFrequencyDict("Плач Иеремии.txt", "S3.txt")

In [14]:
dictLemmatisation3=CreateLemmatisationFrequencyDict("Плач Иеремии.txt","L3.txt")

## Читаем Исповедь Круга

In [15]:
dictStemming4=CreateStemmingFrequencyDict("Исповедь Круга.txt", "S4.txt")

In [16]:
dictLemmatisation4=CreateLemmatisationFrequencyDict("Исповедь Круга.txt","L4.txt")

## Составляем TF-IDF

In [17]:
dictLemmatisationList = [dictLemmatisation1, dictLemmatisation2, dictLemmatisation3, dictLemmatisation4]

In [18]:
dictStemmingList = [dictStemming1, dictStemming2, dictStemming3, dictStemming4]

In [19]:
def mergeDict(sourceDict, newDict):
    
    for key in newDict.keys():
        if key in sourceDict:
            sourceDict[key]+=1
        else:
            sourceDict[key]=1
    
    return sourceDict

In [20]:
def MakeTfIdfArray(dictList):
    
    mergingDict={}
    for dictionary in dictList:
        mergingDict = mergeDict(mergingDict, dictionary)
        
    tf_idf = []
    for element in dictList:
        element_tf_idf=dict()
        totalCount = sum(element.values())
        for word in element:
            termValue = element[word]/totalCount * math.log(len(dictList)/mergingDict[word])
            element_tf_idf[word] = termValue
        tf_idf.append(element_tf_idf)
        
    return tf_idf

In [21]:
tfIdfLemmaArray = MakeTfIdfArray(dictLemmatisationList)

In [22]:
tfIdfStemmingArray = MakeTfIdfArray(dictStemmingList)

In [45]:
tfIdfLemmaArray

[{'прилично': 0.0006917636532534384,
  'братия': 0.0013835273065068768,
  'начать': 0.0027670546130137536,
  'древний': 0.0003458818266267192,
  'склад': 0.0006917636532534384,
  'печальный': 0.0003458818266267192,
  'повесть': 0.0013835273065068768,
  'битва': 0.004150581919520631,
  'игорь': 0.01729409133133596,
  'святославич': 0.006917636532534384,
  'начаться': 0.0006917636532534384,
  'сей': 0.001729409133133596,
  'песня': 0.0024211727863870346,
  'былина': 0.0006917636532534384,
  'сие': 0.00043066178510745636,
  'время': 0.0015790932120606734,
  'вымысел': 0.0006917636532534384,
  'бояновое': 0.0006917636532534384,
  'вещий': 0.003458818266267192,
  'боян': 0.004150581919520631,
  'сотворить': 0.0006917636532534384,
  'хотеть': 0.0007177696418457606,
  'растекаться': 0.0006917636532534384,
  'мыслия': 0.0020752909597603155,
  'древо': 0.004150581919520631,
  'серый': 0.0020752909597603155,
  'волк': 0.006225872879280945,
  'земля': 0.006172818919873541,
  'сизый': 0.0006917636

In [46]:
tfIdfStemmingArray

[{'приличн': 0.000660769476224924,
  'нам': 0.0006856102775304597,
  'брат': 0.005286155809799392,
  'нача': 0.000991154214337386,
  'древн': 0.000330384738112462,
  'склад': 0.000660769476224924,
  'печальн': 0.000330384738112462,
  'повест': 0.001321538952449848,
  'битв': 0.003964616857349544,
  'игор': 0.0165192369056231,
  'святославич': 0.00660769476224924,
  'се': 0.000660769476224924,
  'песн': 0.002312693166787234,
  'былин': 0.000660769476224924,
  'сег': 0.00041136616651827576,
  'времен': 0.00330384738112462,
  'вымысл': 0.000660769476224924,
  'боянов': 0.000660769476224924,
  'вещ': 0.00330384738112462,
  'боя': 0.003964616857349544,
  'есл': 0.0005484882220243676,
  'ком': 0.0001371220555060919,
  'сотвор': 0.000660769476224924,
  'хотел': 0.000660769476224924,
  'растека': 0.000660769476224924,
  'мысл': 0.002643077904899696,
  'древ': 0.003964616857349544,
  'сер': 0.001982308428674772,
  'волк': 0.005946925286024316,
  'земл': 0.005896248386761952,
  'сиз': 0.00066076

## Реферируем, сортируя по стоимости предложений

In [36]:
def MakeAbstract(fileName, tfIdfArray, coef):
    
    fileId = sourceFileNames[fileName]
    sentences = GetSentencesFromFile(fileName)
    sentencesDict = {}
    morph = pymorphy2.MorphAnalyzer()
    
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
    
        sumVal = 0
        for word in words:
            newWord = morph.parse(word.lower())[0].normal_form
            if newWord in tfIdfArray[fileId]:
                sumVal += tfIdfArray[fileId][newWord]
            
        sentencesDict[sentence] = sumVal/len(words)
    
    sortedList = sorted(sentencesDict, key=sentencesDict.get, reverse=True)
    text = sortedList[:round(coef*len(sortedList))]
    finalText = ""
    for sentence in sentencesDict:
        if (sentence in text):
            finalText+=sentence+" ("+ str(sentencesDict[sentence]) +") "+"\n"
    
    return finalText

In [37]:
print(MakeAbstract("Слово о полку.txt", tfIdfLemmaArray, 0.1))

Тогда вступил князь Игорь в златое стремя
И поехал по чистому полю. (0.0037780937985380093) 
Русские поле великое червлеными щитами прегородили,
Ища себе чести, а князю славы. (0.003297406747174723) 
Тут-то копьям поломаться,
Тут-то саблям притупиться
О шеломы половецкие,
На реке на Каяле, у Дона великого! (0.002805543693965274) 
Русские полки отступили. (0.00458293420280403) 
Бесовы дети кликом поля прегородили,
А храбрые русские щитами червлеными. (0.0032113697236612494) 
Ярый тур Всеволод! (0.003285877352953833) 
Игорь полки заворачивает! (0.006398813792594305) 
Где ж ты, великий князь Всеволод! (0.0029497477720492344) 
Вступите, вступите в стремя златое
За честь сего времени, за Русскую землю,
За раны Игоря, буйного Святославича! (0.0028559955692756054) 
Стреляй же, князь, в Кончака, неверного кощея, за Русскую землю,
За раны Игоря, буйного Святославича! (0.003283465809344405) 
Дон тебя, князя, кличет,
Дон зовет князей на победу! (0.004044156742097025) 
Ольговичи, храбрые князи, до

In [38]:
print(MakeAbstract("Слово о полку.txt", tfIdfStemmingArray, 0.1))

Сядем же, други, на борзых коней
Да посмотрим синего Дона!»
Вспала князю на ум охота,
А знаменье заступило ему желание
Отведать Дона великого. (0.000774695247987842) 
Хочу положить свою голову
Или выпить шеломом из Дона». (0.0016218887143702682) 
Тебе бы песнь гласить Игорю, оного Олега внуку:
Не буря соколов занесла чрез поля широкие —
Галки стадами бегут к Дону великому! (0.0008193541505189058) 
И рек ему буй-тур Всеволод:
«Один мне брат, один свет светлый ты, Игорь,
Оба мы Святославичи! (0.0009747140068870584) 
И половцы неготовыми дорогами побежали к Дону великому. (0.0011747012910665316) 
Гзак бежит серым волком,
А Кончак ему след прокладывает к Дону великому! (0.0017935171497533652) 
Идти дождю стрелами с Дону великого! (0.0015103302313712549) 
Тут-то копьям поломаться,
Тут-то саблям притупиться
О шеломы половецкие,
На реке на Каяле, у Дона великого! (0.0010572311619598783) 
Ярый тур Всеволод! (0.002808270273955927) 
Были веки Трояновы,
Миновались лета Ярославовы;
Были битвы Олег

In [39]:
print(MakeAbstract("Поучение Мономаха.txt", tfIdfLemmaArray, 0.05))

Лучше праведнику малое, нежели многие богатства грешным. (0.002027883562483056) 
Ибо сила грешных сокрушится, праведных же укрепляет Господь. (0.0031854243972115753) 
Господом стопы человека направляются. (0.002498605276403435) 
Не забывайте этого, не ленитесь, ибо тем ночным поклоном и молитвой человек побеждает дьявола, и что нагрешит за день, то этим человек избавляется. (0.0018761544689835845) 
Больного навестите, покойника проводите, ибо все мы смертны. (0.0020530110855315606) 



In [40]:
print(MakeAbstract("Поучение Мономаха.txt", tfIdfStemmingArray, 0.05))

Господом стопы человека направляются. (0.0014145860827753985) 
И скажет человек: “Если есть награда праведнику, значит есть Бог, творящий суд на земле”». (0.0006022383265931874) 
И этому чуду подивимся, как из праха создал человека, как разнообразны человеческие лица; если и всех людей собрать, не у всех один облик, но каждый имеет свой облик лица, по Божьей мудрости. (0.0004801088075411569) 
Не забывайте этого, не ленитесь, ибо тем ночным поклоном и молитвой человек побеждает дьявола, и что нагрешит за день, то этим человек избавляется. (0.0006217960803408346) 
Не пропустите человека, не поприветствовав его, и доброе слово ему молвите. (0.000505209315276928) 



In [41]:
print(MakeAbstract("Плач Иеремии.txt", tfIdfLemmaArray, 0.05))

Сион простирает руки свои, но утешителя нет ему. (0.002836891598471809) 
Как помрачил Господь во гневе Своём дщерь Сиона! (0.003937046039249722) 
Господь определил разрушить стену дщери Сиона, протянул вервь, не отклонил руки Своей от разорения; истребил внешние укрепления, и стены вместе разрушены. (0.002472120984783751) 
Сердце их вопиет к Господу: стена дщери Сиона! (0.0035227934744904974) 
Потоки вод изливает око моё о гибели дщери народа моего. (0.0028182116340770345) 
Совершил Господь гнев Свой, излил ярость гнева Своего и зажег на Сионе огонь, который пожрал основания его. (0.0022208452207537446) 
Дщерь Сиона! (0.0072498340849835126) 



In [42]:
print(MakeAbstract("Плач Иеремии.txt", tfIdfStemmingArray, 0.05))

Сион простирает руки свои, но утешителя нет ему. (0.0011719879864540137) 
Как помрачил Господь во гневе Своём дщерь Сиона! (0.001860298391196847) 
Сердце их вопиет к Господу: стена дщери Сиона! (0.0012557014140578717) 
Потоки вод изливает око моё о гибели дщери народа моего. (0.001103495182050857) 
Совершил Господь гнев Свой, излил ярость гнева Своего и зажег на Сионе огонь, который пожрал основания его. (0.0010882745588501556) 
Дщерь Сиона! (0.003906626621513379) 
Жен бесчестят на Сионе, девиц – в городах Иудейских. (0.0012557014140578719) 



In [43]:
print(MakeAbstract("Исповедь Круга.txt", tfIdfStemmingArray, 0.5))

Я сегодня в церкви старой перед батюшкой стою
Взгляд ловлю его усталый, как свеча в огне горю. (0.0005527489478149484) 
Ведь пришёл сегодня к богу я покаяться в грехах
Хоть болит душа немного, только мне неведом страх. (0.000849903326482795) 
Отпусти мне, батюшка, грехи. (0.0015567211712758705) 
В старой церкви пахнет воском, я не в силах промолчать
Совершать грехи так просто, но непросто искупать. (0.0010699639103707557) 
Только батюшка ответил: Кто сегодня без греха? (0.001210783133214566) 
Свой на теле старый крестик я ни разу не снимал
От печали и болезни он не раз меня спасал
Ошибался – было дело, я не раз сходил с пути
Жизнь такая надоела, святый господи, прости. (0.0002692879489354877) 



In [44]:
print(MakeAbstract("Исповедь Круга.txt", tfIdfLemmaArray, 0.5))

Я сегодня в церкви старой перед батюшкой стою
Взгляд ловлю его усталый, как свеча в огне горю. (0.008568454134262212) 
Отпусти мне, батюшка, грехи. (0.009309054127637967) 
Не тая, всю правду расскажу, исповедь мою, прошу, послушай
На алтарь тебе я положу свою искалеченную душу. (0.009818781544841716) 
На алтарь тебе я положу свою искалеченную душу. (0.01172517545858836) 
В старой церкви пахнет воском, я не в силах промолчать
Совершать грехи так просто, но непросто искупать. (0.005772332529557486) 
Только батюшка ответил: Кто сегодня без греха? (0.010008930756262639) 

